# Comparing multivariate analysis using diagonal priors with univariate computations

This is to verify that the mmbr implementation is correct for the truthly multivariate computations. Previously I have only compared it for the degenerated case where the prior matrix is 1 by 1.

## Simulate data

In [1]:
set.seed(1)
n = 1000
p = 1000
X = matrix(rnorm(n*p),nrow=n,ncol=p)
beta1 = beta2 = rep(0,p)
beta1[1:4] = runif(4)
beta2[1:4] = runif(4)
y1 = X %*% beta1 + rnorm(n)
y2 = X %*% beta2 + rnorm(n)

In [2]:
beta1[1:4]

[1] 0.6143060 0.3201991 0.2052207 0.8312957

In [3]:
beta2[1:4]

[1] 0.1468283 0.3689267 0.7327256 0.7734084

## Run univariate computation

In [4]:
m1 = susieR::susie(X,y1,residual_variance = var(y1), estimate_residual_variance=F, scaled_prior_variance = 0.2, L=10)
bhat1 = coef(m1)

In [5]:
m2 = susieR::susie(X,y2,residual_variance = var(y2), estimate_residual_variance=F, scaled_prior_variance = 0.2, L=10)
bhat2 = coef(m2)

In [6]:
bhat1[2:5]

[1] 0.64516270 0.36605187 0.04277906 0.81756620

In [7]:
bhat2[2:5]

[1] 0.09804756 0.38960535 0.73815644 0.73194730

## Run multivariate computation

In [8]:
prior_var = diag(0.2 * c(var(y1), var(y2)))
residual_var = diag(c(var(y1), var(y2)))
y = cbind(y1,y2)
data = mmbr:::DenseData$new(X,y)

In [9]:
prior_covar = mmbr:::MashInitializer$new(list(prior_var), 1, 1, 0)

In [10]:
m = mmbr::susie(X, y, L=10, V=prior_covar)

In [11]:
head(m$coef)

intercept,-1.196833e-02,9.986033e-03
,6.384609e-01,1.821579e-01
,3.624641e-01,3.882791e-01
,1.739475e-01,7.344354e-01
,8.167865e-01,7.304320e-01
,3.458659e-08,1.380051e-07
